RNN
===

#1. 引入RNN
传统的神经网络，包括CNN，输入核输出都是相互独立的。比如输入的是像素点，输出的是图像，基本是没关系的。但是有些任务，后续的输出和之前的内容是相关的，比如我是中国人，我的母语是汉语。这个地方的汉语，跟我是中国人是相关联的。这个时候就需要引入RNN->循环神经网络。
- 循环来源于其么个元素都执行相同的任务
- 输出依赖于输入和"记忆"

#2. 简单结构
![images](images/01.png)
- $X_t$是时间t处的输入
- $S_t$是时间t处的"记忆",$S_t=f(UX_t + WS_{t-1})$, f可以是tanh等
- $O_t$是时间t处的输出，比如是预测下个次的话，可能是softmax输出的属于每个候选词的概率，$O_t=softmax(VS_t)$

##2.1 结构细节
- 可以把隐状态$S_t$视作"记忆体"，波捉了之前时间点上的信息。
- 输出$O_t$由当前时间及之前所有的"记忆"共同计算得到
- $S_t$并不能博主哦核保留之前所有的信息
- 不同于CNN，这里的RNN其实整个神经网络都共享一组参数(U,V,W)，极大减少了需要训练和预估的参数量
- 图中的$O_t$在有些人武侠是不存在的，比如文本情感分析，其实只需要最后的output结果就行了

#3. 不同类型的RNN

##3.1 双向RNN
有些情况下，当前的输出不只依赖于前面的序列元素，还可能依赖之后的序列元素，比如从一段话中踢掉部分词，让你补全。
![images](images/02.png)

##3.2 深层双向RNN
和双向RNN的区别是每一步或每个时间点我们设定多层结构
![images](images/03.png)

#4. RNN与BPTT算法(Back Propagation Through Time)


##4.1 BPTT算法
- 多层神经网络与卷积神经网络都是用BP算法求偏导
- BPTT算法和BP算法是一个思路，只不过既然有step，就和时间t有关系
![images](images/04.png)


其中${x_0, x_1, x_2, x_3, x_4}$是{我，爱，北京，天安门，广场},${E_0, E_1, E_2, E_3}$则是{爱，北京，天安门，广场}.并且我爱北京天安门广场这些词在词典中是有的。其中词典中的词有40000个。

对于任意时间点的输出，我们都可以计算一个交叉熵损失$E_t(y_t,\hat{y_t})=-y_t \bullet log\hat{y_t}$，并且每个时间的输出都是一个40000 * 1的向量，这个向量的每个元素，都是这个位置上的词可能出现在这个时间点的输出的概率，通过softmax可以找到一个最可能的输出，也就是概率最大的那个词，可以组成一个40000 * 1的另外一个向量，这个向量其余位置都是0，只有最可能输出的那个词的位置上是1(one-hot编码).

那么完成一整个过程，就是要把所有时间的损失的输出加起来，$E(y,\hat{y})=\sum_tE_t(y_t,\hat{y_t})=-\sum_ty_t \bullet log\hat{y_t}$。我们的目标就是找到一组W，使得这个损失最小。那么办法就是使用SGD(随机梯度下降)，$W-\eta \bullet \frac{\partial{J(W)}}{\partial{W}}=W-\eta \bullet \frac{\partial{E}}{\partial{W}} = W-\eta \bullet \sum_t\frac{\partial{E_t}}{\partial{W}}$。最核心的就是求每个时间点处的偏导。

假设现在的时间是3，我们求$\frac{\partial{E_3}}{\partial{W}}=\frac{\partial{E_3}}{\partial{\hat{y_3}}} \bullet \frac{\partial{\hat{y_3}}}{\partial{s_3}} \bullet \frac{\partial{s_3}}{\partial{W}}$，但是$s_3=tanh(Ux_t + Ws_2)$，依赖于$s_2$，所以$\frac{\partial{s_3}}{\partial{W}}$这个是没办法求的，因为$s_2$跟W也是有关系的，这是一个复合函数求导，所以还需要把$s_2$继续展开，所以，$\frac{\partial{E_3}}{\partial{W}}=\sum_{k=0}^3\frac{\partial{E_3}}{\partial{\hat{y_3}}} \bullet \frac{\partial{\hat{y_3}}}{\partial{s_3}} \bullet \frac{\partial{s_3}}{\partial{s_k}} \bullet \frac{\partial{s_k}}{\partial{W}}$
![images](images/05.png)

#5. CNN与RNN的区别
- CNN空间扩展，神经元与特征卷积；RNN时间扩展，神经元与多个时间输出计算
- RNN可以用于描述时间上连续状态的输出，有记忆功能，CNN用于静态输出
- CNN可以达到100+的深度，而RNN深度有限

#6. CNN+RNN的组合


##6.1 组合的方式
- 特征提取：LSTM输出，FC层输出
- 特征合并：Concatenate层；Attention相乘
- 结果输出：连续语句输出LSTM，组合分类回归DNN

##6.2 组合的应用方向


###6.2.1 图片标注
CNN提取特征，RNN语句生成。拥有大量图片和标注信息，能否通过学习建立一个能够自动图片标注的模型。最终生成句子的肯定是LSTM，解析图片肯定是CNN模型。两者如何结合？LSTM接收的是一维特征，那么在CNN中产生一维特征的就是全连接层。

比如我们需要对一副图片进行文字描述，可以这样做：
- 首先通过卷积神经网络对图片进行特征提取，比如使用alexnet，最后可以得到一个4096 * 1的向量.
- 在RNN中，之前与$S_t=tanh(W_{xh} \bullet X_t + W_{hh} \bullet S_{t-1})$，那么加入图像特征之后，就变成了$S_t=tanh(W_{xh} \bullet X_t + W_{hh} \bullet S_{t-1} + W_{ih} \bullet V)$，相当于在之前的文本输入以前前一步的记忆的基础上，加入了图像特征矩阵，但是这一步仅仅在RNN的第一步做。相当于看一遍图片，然后开始说话，而不是边看边说

然后进行数据准备，使用图片CNN特征提取，图片标注生成Word2Vect向量，然后生成训练数据：图片特征+第n单词向量：第n+1单词向量;最后进行模型训练，运用迁移学习(FuneTune)生成CNN特征，语句特征应用已有模型，最终输出模型是LSTM，参数设定包括梯度上限，学习率调整

###6.2.2 视频分类
RNN特征提取，用于CNN内容分类


###6.2.3 图片问答
CNN特征提取用于对话问答
